## Полузёрова Анастасия 
## Магистратура ТюмГУ
## Практическое задание 3
# <div style="text-align:center;"><font color="blue">QIIME 2 </font></div>




In [1]:
!conda --version

conda 23.5.0


In [2]:
!ls data_met/sequences

Abacumov-B-13_S13_L001_R1_001.fastq.gz	Abacumov-B-39_S39_L001_R1_001.fastq.gz
Abacumov-B-13_S13_L001_R2_001.fastq.gz	Abacumov-B-39_S39_L001_R2_001.fastq.gz
Abacumov-B-14_S14_L001_R1_001.fastq.gz	Abacumov-B-3_S3_L001_R1_001.fastq.gz
Abacumov-B-14_S14_L001_R2_001.fastq.gz	Abacumov-B-3_S3_L001_R2_001.fastq.gz
Abacumov-B-15_S15_L001_R1_001.fastq.gz	Abacumov-B-40_S40_L001_R1_001.fastq.gz
Abacumov-B-15_S15_L001_R2_001.fastq.gz	Abacumov-B-40_S40_L001_R2_001.fastq.gz
Abacumov-B-16_S16_L001_R1_001.fastq.gz	Abacumov-B-49_S49_L001_R1_001.fastq.gz
Abacumov-B-16_S16_L001_R2_001.fastq.gz	Abacumov-B-49_S49_L001_R2_001.fastq.gz
Abacumov-B-1_S1_L001_R1_001.fastq.gz	Abacumov-B-4_S4_L001_R1_001.fastq.gz
Abacumov-B-1_S1_L001_R2_001.fastq.gz	Abacumov-B-4_S4_L001_R2_001.fastq.gz
Abacumov-B-25_S25_L001_R1_001.fastq.gz	Abacumov-B-50_S50_L001_R1_001.fastq.gz
Abacumov-B-25_S25_L001_R2_001.fastq.gz	Abacumov-B-50_S50_L001_R2_001.fastq.gz
Abacumov-B-26_S26_L001_R1_001.fastq.gz	Abacumov-B-51_S51_L001_R1_001.fastq.g

In [3]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path data_met/sequences \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Imported data_met/sequences as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza


## Deblur

In [4]:
#фильтрация последовательностей на основе оценки качества 
!qiime quality-filter q-score \
 --i-demux demux-paired-end.qza \
 --o-filtered-sequences demux-filtered.qza \
 --o-filter-stats demux-filter-stats.qza

Saved SampleData[SequencesWithQuality] to: demux-filtered.qza
Saved QualityFilterStats to: demux-filter-stats.qza


In [5]:
#с помощью deblur объединяются похожие последовательности и удаляются ошибки секвенирования
!qiime deblur denoise-16S \
  --i-demultiplexed-seqs demux-filtered.qza \
  --p-trim-length 120 \
  --o-representative-sequences rep-seqs-deblur.qza \
  --o-table table-deblur.qza \
  --p-sample-stats \
  --o-stats deblur-stats.qza

Saved FeatureTable[Frequency] to: table-deblur.qza
Saved FeatureData[Sequence] to: rep-seqs-deblur.qza
Saved DeblurStats to: deblur-stats.qza


In [6]:
# табулирование статистики фильтрации для визуализации. 
# Результаты сохраняются в файле demux-filter-stats.qzv.
!qiime metadata tabulate \
  --m-input-file demux-filter-stats.qza \
  --o-visualization demux-filter-stats.qzv
!qiime deblur visualize-stats \
  --i-deblur-stats deblur-stats.qza \
  --o-visualization deblur-stats.qzv

Saved Visualization to: demux-filter-stats.qzv
Saved Visualization to: deblur-stats.qzv


Оценить эффективность фильтрации и качество данных можно с помощь. визуализации полученной таблицы:


![deblur-stats.qzv](Screenshot_20230618_145223.png)


## Add metadata

In [7]:
!sed 's/,/\t/g' map.csv > map.tsv #преобразование файла метаданных map.csv в формат TSV 

In [8]:
!sed -i '1s/Filename/#SampleID/' map.tsv #замена"Filename" на "SampleID"

In [11]:
# Создание сводной информации о таблице признаков
!qiime feature-table summarize \
  --i-table table-deblur.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file map.tsv

# Создание таблицы с информацией о последовательностях (включая идентификаторы и количество вхождений)
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs-deblur.qza \
  --o-visualization rep-seqs.qzv


Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv


![sample-frequencies](sample-frequenc.png)
![feature-frequencies](feature-frequenc.png)
![rep-seqs](rep-seq.png)



## Add tree

In [12]:
# QIIME выравнивает последовательности и строит филогенетическое дерево: 
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs-deblur.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


# Diversity Analysis

Анализ разнообразия микробиома на уровне альфа- и бета-разнообразия 


In [14]:
#QIIME рассчитывает различные метрики альфа- и бета-разнообразия:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table-deblur.qza \
  --p-sampling-depth 8000 \
  --m-metadata-file map.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

Visualise alpha-diversity

In [15]:
!qiime diversity alpha-rarefaction \
  --i-table table-deblur.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 20000 \
  --m-metadata-file map.tsv \
  --o-visualization alpha-rarefaction.qzv

Saved Visualization to: alpha-rarefaction.qzv


![alpha-rarefaction](alpha-rarefaction.png)
![bray-curtis](bray.png)

In [16]:
#статистическая оценка значимости альфа-разнообразия между группами (на основе векторов альфа-разнообразия):
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/observed_features_vector.qza \
  --m-metadata-file map.tsv \
  --o-visualization core-metrics-results/observed_features-significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file map.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv


Saved Visualization to: core-metrics-results/observed_features-significance.qzv
Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


![alpha-diversity-boxplots](alpha-diversity-boxplots.png)
![kruskal-wallis](kruskal-wallis.png)

Тест Kruskal-Wallis:
Нулевая гипотеза в этом тесте заключается в том, что распределения значений во всех группах равны.

На основании полученных результатов можно сделать вывод, что существует статистически значимая разница между группами. Значение p-value меньше уровня значимости 0.05, что позволяет отвергнуть нулевую гипотезу. Это означает, что распределения значений в разных группах статистически отличаются, и существует вероятность, что эти различия не случайны.

Между группами "Coal Mine Terricon" и "Embryo Sand" наблюдается статистически значимая разница (p-value = 0.020921), с учетом поправки на множественные сравнения (q-value = 0.024140).
Между группами "Litostrat" и "Embryo Sand" также наблюдается статистически значимая разница (p-value = 0.083265), но после коррекции на множественные сравнения эта разница становится менее значимой (q-value = 0.089212).
Между другими парами групп также наблюдаются значения статистики H, p-value и q-value, которые указывают на различия между группами, но точность этих различий может быть несколько ниже из-за коррекции на множественные сравнения.

In [17]:
# Beta-diversity significance
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file map.tsv \
  --m-metadata-column Source\
  --o-visualization core-metrics-results/bray_curtis-body-site-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/bray_curtis-body-site-significance.qzv


![bray_curtis_body_site_significance1](bray_curtis_body_site_significance1.png)
![bray_curtis_body_site_significance2](bray_curtis_body_site_significance2.png)
![permanova_results](permanova_results.png)

Анализ PERMANOVA подтверждает, что существует статистически значимая разница между группами на основе анализа многомерных данных.